In [17]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import lightgbm

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score


# set options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


vote_df = pd.read_csv("C:\\Users\\leejingang\\PycharmProjects\\AI\\train.csv", encoding="utf-8")
print("dataset shape: ", vote_df.shape)

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred) # 오차 행렬
    accuracy = accuracy_score(y_test, pred) # 정확도
    precision = precision_score(y_test, pred) # 정밀도
    recall = recall_score(y_test, pred) # 재현율
    f1 = f1_score(y_test, pred) # F1
    roc_auc = roc_auc_score(y_test, pred_proba) # ROC-AUC
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))



# 인덱스 제거
vote_df.drop("index", axis=1, inplace=True)

# 0값을 검사할 피처명 리스트 객체 설정
zero_features = ['education', 'engnat', 'hand', 'urban']

# tp 추가
for i in range(1, 11):
    s = 'tp' + chr(48 + (i // 10)) + chr(48 + (i % 10))
    zero_features.append(s)

# zero_features 리스트 내부에 저장된 개별 피처들에 대해 0값을 평균 값으로 대체
vote_df[zero_features] = vote_df[zero_features].replace(0, vote_df[zero_features].mean())



Q_E = []
#Q_E 시간
for i in range(97, 117):
    a = 'Q' + chr(i) + 'E'
    Q_E.append(a)


# 시간 데이터 전처리 - 정확도 0.85 -> 0.94로 증가함 하지만 StandardScaler로 전처리는 의미가 없을 걸로 보임...
def get_categoty_time(Q_time):
    cat = 0
    if Q_time <= 1000 : cat = 1
    elif Q_time <= 2000 : cat = 2
    elif Q_time <= 3000 : cat = 3
    elif Q_time <= 4000 : cat = 4
    elif Q_time <= 5000 : cat = 5
    elif Q_time <= 6000 : cat = 6
    elif Q_time <= 7000 : cat = 7
    elif Q_time <= 8000 : cat = 8
    elif Q_time <= 9000 : cat = 9
    elif Q_time <=10000 : cat = 10
    elif Q_time <=15000 : cat = 11
    elif Q_time <=20000 : cat = 12
    else : Q_time = 13

    return cat


for Q_time in Q_E:
    vote_df[Q_time] = vote_df[Q_time].apply(lambda x : get_categoty_time(x))


# 시간에 대해 이상치 데이터 인덱스 확인 및 제거
def get_outlier(df=None, column=None, weight=1.5):
    fraud = df[df['voted'] == 1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)

    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index


for feature in Q_E:
    outlier_index = get_outlier(df=vote_df, column=feature, weight=1.5)
    print('이상치 데이터 인덱스 :', outlier_index)
    vote_df.drop(outlier_index, axis=0, inplace=True)


# 레이블 인코딩
encoder = LabelEncoder()
encoder.fit(vote_df['age_group'])
labels = encoder.transform(vote_df['age_group'])
vote_df['age_group'] = labels



def get_categoty_age(age_num):
    num = age_num;
    if age_num == 0: num =7

    return num
vote_df["age_group"] = vote_df["age_group"].apply(lambda x : get_categoty_age(x))
# x_test["age_group"] = x_test["age_group"].apply(lambda x : get_categoty_age(x))



#원핫인코딩 진행
def dummy_data(data, columns) :
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data

dummy_columns = ['engnat', 'gender','hand', 'married', 'race', 'religion', 'urban']
vote_df = dummy_data(vote_df, dummy_columns)



# X_features, y_labels 설정
y_labels = vote_df.loc[:,'voted']
X_features = vote_df.drop('voted', axis=1)
print("피처 데이터 shape:{0}".format(X_features.shape))



# Standard Scaler 적용
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
vote_scaled = scaler.fit_transform(X_features)

X_features = pd.DataFrame(data=vote_scaled, columns=X_features.columns.values.tolist())


# train and test data split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=156)
train_cnt = y_train.count()
test_cnt = y_test.count()
print("학습 세트 Shape:{0}, 테스트 세트 Shape:{1}".format(X_train.shape, X_test.shape))

print(" 학습 세트 레이블 값 분포 비율")
print(y_train.value_counts()/train_cnt)
print("\n 테스트 세트 레이블 값 분포 비율")
print(y_test.value_counts()/test_cnt)


from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트:', X_train_over.shape, y_train_over.shape)
print()
print('SMOTE 적용 후 레이블 값 분포:')
print(y_train_over.value_counts())

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score



dataset shape:  (36425, 78)
이상치 데이터 인덱스 : Int64Index([    6,    21,    22,    27,    31,    33,    35,    49,    60,
               73,
            ...
            36221, 36235, 36247, 36252, 36295, 36336, 36350, 36363, 36373,
            36424],
           dtype='int64', length=3118)
이상치 데이터 인덱스 : Int64Index([   43,    90,   143,   182,   190,   194,   250,   263,   377,
              450,
            ...
            36183, 36204, 36254, 36267, 36274, 36306, 36320, 36322, 36364,
            36387],
           dtype='int64', length=1001)
이상치 데이터 인덱스 : Int64Index([   96,   101,   131,   270,   273,   346,   409,   424,   442,
              578,
            ...
            35872, 36125, 36155, 36201, 36237, 36290, 36304, 36325, 36358,
            36365],
           dtype='int64', length=406)
이상치 데이터 인덱스 : Int64Index([    9,    66,   258,   266,   274,   400,   422,   650,   743,
              750,
            ...
            35507, 35563, 35628, 35759, 35822, 35916, 35927, 36224, 36374,


In [18]:
# n_estimators는 500으로, random state는 예제 수행 시마다 동일 예측 결과를 위해 설정. 
xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

# 성능 평가 지표를 auc로, 조기 중단 파라미터는 100으로 설정하고 학습 수행. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
            eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))


[0]	validation_0-auc:0.94877	validation_1-auc:0.94386
[1]	validation_0-auc:0.95206	validation_1-auc:0.94592
[2]	validation_0-auc:0.95793	validation_1-auc:0.95254
[3]	validation_0-auc:0.95885	validation_1-auc:0.95312
[4]	validation_0-auc:0.96077	validation_1-auc:0.95481
[5]	validation_0-auc:0.96288	validation_1-auc:0.95783
[6]	validation_0-auc:0.96341	validation_1-auc:0.95860
[7]	validation_0-auc:0.96598	validation_1-auc:0.96247
[8]	validation_0-auc:0.96751	validation_1-auc:0.96456
[9]	validation_0-auc:0.96946	validation_1-auc:0.96661
[10]	validation_0-auc:0.96997	validation_1-auc:0.96741
[11]	validation_0-auc:0.97046	validation_1-auc:0.96824


C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12]	validation_0-auc:0.97115	validation_1-auc:0.96905
[13]	validation_0-auc:0.97211	validation_1-auc:0.96954
[14]	validation_0-auc:0.97219	validation_1-auc:0.96966
[15]	validation_0-auc:0.97256	validation_1-auc:0.97015
[16]	validation_0-auc:0.97310	validation_1-auc:0.97084
[17]	validation_0-auc:0.97369	validation_1-auc:0.97131
[18]	validation_0-auc:0.97382	validation_1-auc:0.97142
[19]	validation_0-auc:0.97477	validation_1-auc:0.97180
[20]	validation_0-auc:0.97500	validation_1-auc:0.97215
[21]	validation_0-auc:0.97518	validation_1-auc:0.97247
[22]	validation_0-auc:0.97643	validation_1-auc:0.97191
[23]	validation_0-auc:0.97666	validation_1-auc:0.97235
[24]	validation_0-auc:0.97693	validation_1-auc:0.97244
[25]	validation_0-auc:0.97711	validation_1-auc:0.97273
[26]	validation_0-auc:0.97857	validation_1-auc:0.97214
[27]	validation_0-auc:0.97870	validation_1-auc:0.97237
[28]	validation_0-auc:0.97891	validation_1-auc:0.97261
[29]	validation_0-auc:0.97906	validation_1-auc:0.97254
[30]	valid

In [19]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=100)

params = {'max_depth':[5, 7] , 'min_child_weight':[1,3] ,'colsample_bytree':[0.5, 0.75] }

# 하이퍼 파라미터 테스트의 수행속도를 향상 시키기 위해 cv 를 지정하지 않음. 
gridcv = GridSearchCV(xgb_clf, param_grid=params)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:',gridcv.best_params_) 

xgb_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.85412	validation_1-auc:0.85210
[1]	validation_0-auc:0.92924	validation_1-auc:0.92108
[2]	validation_0-auc:0.93552	validation_1-auc:0.92818
[3]	validation_0-auc:0.94950	validation_1-auc:0.94374
[4]	validation_0-auc:0.95243	validation_1-auc:0.94669
[5]	validation_0-auc:0.95757	validation_1-auc:0.95345
[6]	validation_0-auc:0.96207	validation_1-auc:0.95746
[7]	validation_0-auc:0.96343	validation_1-auc:0.95823
[8]	validation_0-auc:0.96605	validation_1-auc:0.96195
[9]	validation_0-auc:0.96684	validation_1-auc:0.96334
[10]	validation_0-auc:0.96708	validation_1-auc:0.96289
[11]	validation_0-auc:0.96855	validation_1-auc:0.96469
[12]	validation_0-auc:0.96983	validation_1-auc:0.96640
[13]	validation_0-auc:0.97020	validation_1-auc:0.96676
[14]	validation_0-auc:0.97096	validation_1-auc:0.96758
[15]	validation_0-auc:0.97131	validation_1-auc:0.96812
[16]	validation_0-auc:0.97171	validation_1-auc:0.96860
[17]	validation_0-auc:0.97182	validation_1-auc:0.96881
[18]	validation_0-au

C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19]	validation_0-auc:0.97242	validation_1-auc:0.96974
[20]	validation_0-auc:0.97348	validation_1-auc:0.97004
[21]	validation_0-auc:0.97373	validation_1-auc:0.97044
[22]	validation_0-auc:0.97394	validation_1-auc:0.97077
[23]	validation_0-auc:0.97423	validation_1-auc:0.97133
[24]	validation_0-auc:0.97437	validation_1-auc:0.97159
[25]	validation_0-auc:0.97453	validation_1-auc:0.97189
[26]	validation_0-auc:0.97467	validation_1-auc:0.97196
[27]	validation_0-auc:0.97478	validation_1-auc:0.97215
[28]	validation_0-auc:0.97489	validation_1-auc:0.97225
[29]	validation_0-auc:0.97576	validation_1-auc:0.97228
[30]	validation_0-auc:0.97666	validation_1-auc:0.97249
[31]	validation_0-auc:0.97790	validation_1-auc:0.97169
[32]	validation_0-auc:0.97796	validation_1-auc:0.97179
[33]	validation_0-auc:0.97810	validation_1-auc:0.97198
[34]	validation_0-auc:0.97820	validation_1-auc:0.97219
[35]	validation_0-auc:0.97896	validation_1-auc:0.97192
[36]	validation_0-auc:0.97911	validation_1-auc:0.97202
[37]	valid

C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18]	validation_0-auc:0.97274	validation_1-auc:0.96790
[19]	validation_0-auc:0.97294	validation_1-auc:0.96829
[20]	validation_0-auc:0.97379	validation_1-auc:0.96860
[21]	validation_0-auc:0.97397	validation_1-auc:0.96899
[22]	validation_0-auc:0.97434	validation_1-auc:0.96926
[23]	validation_0-auc:0.97467	validation_1-auc:0.96973
[24]	validation_0-auc:0.97482	validation_1-auc:0.96982
[25]	validation_0-auc:0.97562	validation_1-auc:0.97026
[26]	validation_0-auc:0.97570	validation_1-auc:0.97040
[27]	validation_0-auc:0.97611	validation_1-auc:0.97066
[28]	validation_0-auc:0.97635	validation_1-auc:0.97080
[29]	validation_0-auc:0.97666	validation_1-auc:0.97100
[30]	validation_0-auc:0.97738	validation_1-auc:0.97089
[31]	validation_0-auc:0.97780	validation_1-auc:0.97113
[32]	validation_0-auc:0.97788	validation_1-auc:0.97126
[33]	validation_0-auc:0.97802	validation_1-auc:0.97146
[34]	validation_0-auc:0.97841	validation_1-auc:0.97157
[35]	validation_0-auc:0.97873	validation_1-auc:0.97169
[36]	valid

...

[15]	validation_0-auc:0.97423	validation_1-auc:0.97116
[16]	validation_0-auc:0.97470	validation_1-auc:0.97164
[17]	validation_0-auc:0.97511	validation_1-auc:0.97189
[18]	validation_0-auc:0.97528	validation_1-auc:0.97217
[19]	validation_0-auc:0.97711	validation_1-auc:0.97191
[20]	validation_0-auc:0.97754	validation_1-auc:0.97190
[21]	validation_0-auc:0.97765	validation_1-auc:0.97214
[22]	validation_0-auc:0.97791	validation_1-auc:0.97245
[23]	validation_0-auc:0.97815	validation_1-auc:0.97266
[24]	validation_0-auc:0.97836	validation_1-auc:0.97292
[25]	validation_0-auc:0.97950	validation_1-auc:0.97304
[26]	validation_0-auc:0.98002	validation_1-auc:0.97277
[27]	validation_0-auc:0.98084	validation_1-auc:0.97294
[28]	validation_0-auc:0.98123	validation_1-auc:0.97296
[29]	validation_0-auc:0.98209	validation_1-auc:0.97255
[30]	validation_0-auc:0.98334	validation_1-auc:0.97225
[31]	validation_0-auc:0.98450	validation_1-auc:0.97192
[32]	validation_0-auc:0.98522	validation_1-auc:0.97186
[33]	valid

C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14]	validation_0-auc:0.97430	validation_1-auc:0.97130
[15]	validation_0-auc:0.97543	validation_1-auc:0.97067
[16]	validation_0-auc:0.97567	validation_1-auc:0.97108
[17]	validation_0-auc:0.97584	validation_1-auc:0.97133
[18]	validation_0-auc:0.97632	validation_1-auc:0.97153
[19]	validation_0-auc:0.97689	validation_1-auc:0.97157
[20]	validation_0-auc:0.97699	validation_1-auc:0.97170
[21]	validation_0-auc:0.97849	validation_1-auc:0.97215
[22]	validation_0-auc:0.97859	validation_1-auc:0.97235
[23]	validation_0-auc:0.97920	validation_1-auc:0.97168
[24]	validation_0-auc:0.97929	validation_1-auc:0.97169
[25]	validation_0-auc:0.97991	validation_1-auc:0.97159
[26]	validation_0-auc:0.98090	validation_1-auc:0.97118
[27]	validation_0-auc:0.98127	validation_1-auc:0.97123
[28]	validation_0-auc:0.98252	validation_1-auc:0.97069
[29]	validation_0-auc:0.98277	validation_1-auc:0.97069
[30]	validation_0-auc:0.98411	validation_1-auc:0.97114
[31]	validation_0-auc:0.98423	validation_1-auc:0.97103
[32]	valid

C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14]	validation_0-auc:0.97094	validation_1-auc:0.96920
[15]	validation_0-auc:0.97137	validation_1-auc:0.96955
[16]	validation_0-auc:0.97163	validation_1-auc:0.96997
[17]	validation_0-auc:0.97197	validation_1-auc:0.97035
[18]	validation_0-auc:0.97228	validation_1-auc:0.97064
[19]	validation_0-auc:0.97274	validation_1-auc:0.97120
[20]	validation_0-auc:0.97298	validation_1-auc:0.97159
[21]	validation_0-auc:0.97307	validation_1-auc:0.97169
[22]	validation_0-auc:0.97323	validation_1-auc:0.97197
[23]	validation_0-auc:0.97390	validation_1-auc:0.97232
[24]	validation_0-auc:0.97427	validation_1-auc:0.97254
[25]	validation_0-auc:0.97443	validation_1-auc:0.97272
[26]	validation_0-auc:0.97457	validation_1-auc:0.97292
[27]	validation_0-auc:0.97493	validation_1-auc:0.97258
[28]	validation_0-auc:0.97614	validation_1-auc:0.97271
[29]	validation_0-auc:0.97691	validation_1-auc:0.97314
[30]	validation_0-auc:0.97706	validation_1-auc:0.97331
[31]	validation_0-auc:0.97719	validation_1-auc:0.97353
[32]	valid

In [20]:

# n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소, reg_alpha=0.03으로 추가함. 
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=5,\
                        min_child_weight=1, colsample_bytree=0.75, reg_alpha=0.03)

# evaluation metric을 auc로, early stopping은 200 으로 설정하고 학습 수행. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, 
            eval_metric="auc",eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.93505	validation_1-auc:0.92975
[1]	validation_0-auc:0.94259	validation_1-auc:0.93659
[2]	validation_0-auc:0.94834	validation_1-auc:0.94388
[3]	validation_0-auc:0.95342	validation_1-auc:0.94717
[4]	validation_0-auc:0.95461	validation_1-auc:0.94899
[5]	validation_0-auc:0.95278	validation_1-auc:0.94682
[6]	validation_0-auc:0.95270	validation_1-auc:0.94675
[7]	validation_0-auc:0.95340	validation_1-auc:0.94730
[8]	validation_0-auc:0.95484	validation_1-auc:0.94916
[9]	validation_0-auc:0.95511	validation_1-auc:0.94918
[10]	validation_0-auc:0.95495	validation_1-auc:0.94885
[11]	validation_0-auc:0.95558	validation_1-auc:0.94986
[12]	validation_0-auc:0.95593	validation_1-auc:0.95046
[13]	validation_0-auc:0.95622	validation_1-auc:0.95080


C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14]	validation_0-auc:0.95668	validation_1-auc:0.95132
[15]	validation_0-auc:0.95729	validation_1-auc:0.95198
[16]	validation_0-auc:0.95583	validation_1-auc:0.95043
[17]	validation_0-auc:0.95497	validation_1-auc:0.94931
[18]	validation_0-auc:0.95282	validation_1-auc:0.94619
[19]	validation_0-auc:0.95587	validation_1-auc:0.94966
[20]	validation_0-auc:0.95616	validation_1-auc:0.94979
[21]	validation_0-auc:0.95722	validation_1-auc:0.95132
[22]	validation_0-auc:0.95746	validation_1-auc:0.95156
[23]	validation_0-auc:0.95829	validation_1-auc:0.95174
[24]	validation_0-auc:0.95941	validation_1-auc:0.95349
[25]	validation_0-auc:0.95985	validation_1-auc:0.95424
[26]	validation_0-auc:0.95999	validation_1-auc:0.95439
[27]	validation_0-auc:0.95938	validation_1-auc:0.95338
[28]	validation_0-auc:0.95916	validation_1-auc:0.95323
[29]	validation_0-auc:0.95952	validation_1-auc:0.95382
[30]	validation_0-auc:0.95893	validation_1-auc:0.95301
[31]	validation_0-auc:0.95868	validation_1-auc:0.95301
[32]	valid

[162]	validation_0-auc:0.96825	validation_1-auc:0.96534
[163]	validation_0-auc:0.96827	validation_1-auc:0.96535
[164]	validation_0-auc:0.96833	validation_1-auc:0.96545
[165]	validation_0-auc:0.96835	validation_1-auc:0.96549
[166]	validation_0-auc:0.96837	validation_1-auc:0.96559
[167]	validation_0-auc:0.96841	validation_1-auc:0.96565
[168]	validation_0-auc:0.96842	validation_1-auc:0.96567
[169]	validation_0-auc:0.96846	validation_1-auc:0.96573
[170]	validation_0-auc:0.96851	validation_1-auc:0.96581
[171]	validation_0-auc:0.96861	validation_1-auc:0.96596
[172]	validation_0-auc:0.96868	validation_1-auc:0.96608
[173]	validation_0-auc:0.96872	validation_1-auc:0.96617
[174]	validation_0-auc:0.96898	validation_1-auc:0.96643
[175]	validation_0-auc:0.96899	validation_1-auc:0.96657
[176]	validation_0-auc:0.96908	validation_1-auc:0.96671
[177]	validation_0-auc:0.96915	validation_1-auc:0.96680
[178]	validation_0-auc:0.96918	validation_1-auc:0.96684
[179]	validation_0-auc:0.96940	validation_1-auc:

[309]	validation_0-auc:0.97347	validation_1-auc:0.97095
[310]	validation_0-auc:0.97350	validation_1-auc:0.97097
[311]	validation_0-auc:0.97352	validation_1-auc:0.97099
[312]	validation_0-auc:0.97352	validation_1-auc:0.97100
[313]	validation_0-auc:0.97354	validation_1-auc:0.97103
[314]	validation_0-auc:0.97356	validation_1-auc:0.97103
[315]	validation_0-auc:0.97356	validation_1-auc:0.97105
[316]	validation_0-auc:0.97358	validation_1-auc:0.97107
[317]	validation_0-auc:0.97359	validation_1-auc:0.97109
[318]	validation_0-auc:0.97361	validation_1-auc:0.97110
[319]	validation_0-auc:0.97364	validation_1-auc:0.97109
[320]	validation_0-auc:0.97366	validation_1-auc:0.97111
[321]	validation_0-auc:0.97368	validation_1-auc:0.97112
[322]	validation_0-auc:0.97370	validation_1-auc:0.97116
[323]	validation_0-auc:0.97379	validation_1-auc:0.97111
[324]	validation_0-auc:0.97388	validation_1-auc:0.97123
[325]	validation_0-auc:0.97381	validation_1-auc:0.97133
[326]	validation_0-auc:0.97383	validation_1-auc:

[456]	validation_0-auc:0.97586	validation_1-auc:0.97316
[457]	validation_0-auc:0.97589	validation_1-auc:0.97317
[458]	validation_0-auc:0.97589	validation_1-auc:0.97318
[459]	validation_0-auc:0.97595	validation_1-auc:0.97319
[460]	validation_0-auc:0.97597	validation_1-auc:0.97319
[461]	validation_0-auc:0.97597	validation_1-auc:0.97320
[462]	validation_0-auc:0.97603	validation_1-auc:0.97322
[463]	validation_0-auc:0.97604	validation_1-auc:0.97319
[464]	validation_0-auc:0.97605	validation_1-auc:0.97319
[465]	validation_0-auc:0.97607	validation_1-auc:0.97319
[466]	validation_0-auc:0.97607	validation_1-auc:0.97319
[467]	validation_0-auc:0.97610	validation_1-auc:0.97321
[468]	validation_0-auc:0.97610	validation_1-auc:0.97322
[469]	validation_0-auc:0.97611	validation_1-auc:0.97323
[470]	validation_0-auc:0.97611	validation_1-auc:0.97324
[471]	validation_0-auc:0.97612	validation_1-auc:0.97323
[472]	validation_0-auc:0.97612	validation_1-auc:0.97324
[473]	validation_0-auc:0.97617	validation_1-auc:

[603]	validation_0-auc:0.97815	validation_1-auc:0.97388
[604]	validation_0-auc:0.97816	validation_1-auc:0.97388
[605]	validation_0-auc:0.97817	validation_1-auc:0.97387
[606]	validation_0-auc:0.97818	validation_1-auc:0.97388
[607]	validation_0-auc:0.97821	validation_1-auc:0.97386
[608]	validation_0-auc:0.97824	validation_1-auc:0.97385
[609]	validation_0-auc:0.97827	validation_1-auc:0.97387
[610]	validation_0-auc:0.97828	validation_1-auc:0.97389
[611]	validation_0-auc:0.97829	validation_1-auc:0.97389
[612]	validation_0-auc:0.97829	validation_1-auc:0.97389
[613]	validation_0-auc:0.97830	validation_1-auc:0.97389
[614]	validation_0-auc:0.97831	validation_1-auc:0.97391
[615]	validation_0-auc:0.97834	validation_1-auc:0.97391
[616]	validation_0-auc:0.97834	validation_1-auc:0.97392
[617]	validation_0-auc:0.97834	validation_1-auc:0.97392
[618]	validation_0-auc:0.97835	validation_1-auc:0.97392
[619]	validation_0-auc:0.97837	validation_1-auc:0.97392
[620]	validation_0-auc:0.97843	validation_1-auc:

[750]	validation_0-auc:0.98016	validation_1-auc:0.97395
[751]	validation_0-auc:0.98016	validation_1-auc:0.97396
[752]	validation_0-auc:0.98017	validation_1-auc:0.97396
[753]	validation_0-auc:0.98017	validation_1-auc:0.97397
[754]	validation_0-auc:0.98018	validation_1-auc:0.97397
[755]	validation_0-auc:0.98018	validation_1-auc:0.97397
[756]	validation_0-auc:0.98018	validation_1-auc:0.97398
[757]	validation_0-auc:0.98018	validation_1-auc:0.97398
[758]	validation_0-auc:0.98020	validation_1-auc:0.97398
[759]	validation_0-auc:0.98022	validation_1-auc:0.97398
[760]	validation_0-auc:0.98022	validation_1-auc:0.97398
[761]	validation_0-auc:0.98025	validation_1-auc:0.97398
[762]	validation_0-auc:0.98027	validation_1-auc:0.97397
[763]	validation_0-auc:0.98032	validation_1-auc:0.97394
[764]	validation_0-auc:0.98033	validation_1-auc:0.97395
[765]	validation_0-auc:0.98034	validation_1-auc:0.97396
[766]	validation_0-auc:0.98034	validation_1-auc:0.97397
[767]	validation_0-auc:0.98039	validation_1-auc:

[897]	validation_0-auc:0.98219	validation_1-auc:0.97396
[898]	validation_0-auc:0.98220	validation_1-auc:0.97396
[899]	validation_0-auc:0.98221	validation_1-auc:0.97396
[900]	validation_0-auc:0.98221	validation_1-auc:0.97396
[901]	validation_0-auc:0.98222	validation_1-auc:0.97396
[902]	validation_0-auc:0.98222	validation_1-auc:0.97396
[903]	validation_0-auc:0.98226	validation_1-auc:0.97395
[904]	validation_0-auc:0.98228	validation_1-auc:0.97395
[905]	validation_0-auc:0.98228	validation_1-auc:0.97395
[906]	validation_0-auc:0.98228	validation_1-auc:0.97394
[907]	validation_0-auc:0.98228	validation_1-auc:0.97395
[908]	validation_0-auc:0.98228	validation_1-auc:0.97394
[909]	validation_0-auc:0.98229	validation_1-auc:0.97394
[910]	validation_0-auc:0.98229	validation_1-auc:0.97393
[911]	validation_0-auc:0.98231	validation_1-auc:0.97394
[912]	validation_0-auc:0.98231	validation_1-auc:0.97393
[913]	validation_0-auc:0.98233	validation_1-auc:0.97393
[914]	validation_0-auc:0.98233	validation_1-auc:

In [21]:

bayes_params = {
    'num_leaves': (24, 45),
    'colsample_bytree':(0.5, 1), 
    'subsample': (0.5, 1),
    'max_depth': (4, 12),
    'reg_alpha': (0, 0.5),
    'reg_lambda': (0, 0.5), 
    'min_split_gain': (0.001, 0.1),
    'min_child_weight':(5, 50)
}

In [22]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

def lgb_roc_eval(num_leaves, colsample_bytree, subsample, max_depth, reg_alpha, reg_lambda, min_split_gain, min_child_weight):
    
    params = {
        "n_estimator":200,
        "learning_rate":0.02,
        'num_leaves': int(round(num_leaves)),
        'colsample_bytree': colsample_bytree, 
        'subsample': subsample,
        'max_depth': int(round(max_depth)),
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda, 
        'min_split_gain': min_split_gain,
        'min_child_weight': min_child_weight,
        'verbosity': -1
    }
    print("params:", params)
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=30, eval_metric="auc", verbose=100 )
    best_iter = lgb_model.best_iteration_
    print('best_iter:', best_iter)
    valid_proba = lgb_model.predict_proba(X_test, num_iteration=best_iter)[:, 1]
    print('roc_auc:', roc_preds)
    return roc_preds

In [14]:
from bayes_opt import BayesianOptimization

BO_lgb = BayesianOptimization(lgb_roc_eval, bayes_params, random_state=0)

In [16]:
BO_lgb.maximize(init_points=5, n_iter=10)

|   iter    |  target   | colsam... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
params: {'n_estimator': 200, 'learning_rate': 0.02, 'num_leaves': 38, 'colsample_bytree': 0.679753950286893, 'subsample': 0.5644631488274267, 'max_depth': 7, 'reg_alpha': 0.3353189348090797, 'reg_lambda': 0.10519128053692045, 'min_split_gain': 0.006962321691297714, 'min_child_weight': 36.39340381672692, 'verbosity': -1}
[LightGBM] [Warning] Unknown parameter: n_estimator
Training until validation scores don't improve for 30 rounds
[100]	valid_0's auc: 0.96285	valid_0's binary_logloss: 0.130084
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.96285	valid_0's binary_logloss: 0.130084
best_iter: 100
roc_auc: 0.9628495817071522
|  16       |  0.9628   |  0.6798   |  7.496    |  36.39    |  0.006962 |  38.0     |  0.3353   |  0.1

best_iter: 100
roc_auc: 0.9724714510197174
|  25       |  0.9725   |  1.0      |  12.0     |  5.0      |  0.1      |  40.92    |  0.5      |  0.5      |  1.0      |
params: {'n_estimator': 200, 'learning_rate': 0.02, 'num_leaves': 45, 'colsample_bytree': 1.0, 'subsample': 1.0, 'max_depth': 12, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'min_split_gain': 0.001, 'min_child_weight': 22.71172764168317, 'verbosity': -1}
[LightGBM] [Warning] Unknown parameter: n_estimator
Training until validation scores don't improve for 30 rounds
[100]	valid_0's auc: 0.967368	valid_0's binary_logloss: 0.120232
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.967368	valid_0's binary_logloss: 0.120232
best_iter: 100
roc_auc: 0.9673680347165007
|  26       |  0.9674   |  1.0      |  12.0     |  22.71    |  0.001    |  45.0     |  0.0      |  0.0      |  1.0      |
params: {'n_estimator': 200, 'learning_rate': 0.02, 'num_leaves': 28, 'colsample_bytree': 1.0, 'subsample': 0.5, 'max_depth': 12, '

In [23]:
BO_lgb.max

{'target': 0.9724714510197174,
 'params': {'colsample_bytree': 1.0,
  'max_depth': 12.0,
  'min_child_weight': 5.0,
  'min_split_gain': 0.1,
  'num_leaves': 40.91732192323167,
  'reg_alpha': 0.5,
  'reg_lambda': 0.5,
  'subsample': 1.0}}

In [25]:
max_params = BO_lgb.max['params']

max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['max_depth'] = int(round(max_params['max_depth']))

xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.02,**max_params)

evals = [(X_test, y_test)]
xgb_model.fit(X_train_over, y_train_over, early_stopping_rounds=100, eval_metric="logloss", eval_set=evals, verbose=True)

w_preds = xgb_model.predict(X_test)
w_pred_proba = xgb_model.predict_proba(X_test)[:,1]

get_clf_eval(y_test, w_preds, w_pred_proba)



[02:19:29] WARNING: ..\src\learner.cc:541: 
Parameters: { min_split_gain, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-logloss:0.67657
[1]	validation_0-logloss:0.66061
[2]	validation_0-logloss:0.64535


C:\Users\leejingang\anaconda3\envs\ssu\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[3]	validation_0-logloss:0.63067
[4]	validation_0-logloss:0.61648
[5]	validation_0-logloss:0.60297
[6]	validation_0-logloss:0.58985
[7]	validation_0-logloss:0.57731
[8]	validation_0-logloss:0.56523
[9]	validation_0-logloss:0.55348
[10]	validation_0-logloss:0.54231
[11]	validation_0-logloss:0.53137
[12]	validation_0-logloss:0.52092
[13]	validation_0-logloss:0.51074
[14]	validation_0-logloss:0.50097
[15]	validation_0-logloss:0.49156
[16]	validation_0-logloss:0.48244
[17]	validation_0-logloss:0.47352
[18]	validation_0-logloss:0.46505
[19]	validation_0-logloss:0.45676
[20]	validation_0-logloss:0.44879
[21]	validation_0-logloss:0.44105
[22]	validation_0-logloss:0.43356
[23]	validation_0-logloss:0.42621
[24]	validation_0-logloss:0.41911
[25]	validation_0-logloss:0.41219
[26]	validation_0-logloss:0.40556
[27]	validation_0-logloss:0.39909
[28]	validation_0-logloss:0.39272
[29]	validation_0-logloss:0.38653
[30]	validation_0-logloss:0.38062
[31]	validation_0-logloss:0.37478
[32]	validation_0-log

[241]	validation_0-logloss:0.11361
[242]	validation_0-logloss:0.11348
[243]	validation_0-logloss:0.11333
[244]	validation_0-logloss:0.11318
[245]	validation_0-logloss:0.11306
[246]	validation_0-logloss:0.11291
[247]	validation_0-logloss:0.11276
[248]	validation_0-logloss:0.11265
[249]	validation_0-logloss:0.11256
[250]	validation_0-logloss:0.11246
[251]	validation_0-logloss:0.11236
[252]	validation_0-logloss:0.11225
[253]	validation_0-logloss:0.11215
[254]	validation_0-logloss:0.11203
[255]	validation_0-logloss:0.11193
[256]	validation_0-logloss:0.11181
[257]	validation_0-logloss:0.11170
[258]	validation_0-logloss:0.11161
[259]	validation_0-logloss:0.11152
[260]	validation_0-logloss:0.11142
[261]	validation_0-logloss:0.11132
[262]	validation_0-logloss:0.11125
[263]	validation_0-logloss:0.11116
[264]	validation_0-logloss:0.11104
[265]	validation_0-logloss:0.11099
[266]	validation_0-logloss:0.11090
[267]	validation_0-logloss:0.11083
[268]	validation_0-logloss:0.11075
[269]	validation_0-l

[476]	validation_0-logloss:0.10706
[477]	validation_0-logloss:0.10708
[478]	validation_0-logloss:0.10711
[479]	validation_0-logloss:0.10709
[480]	validation_0-logloss:0.10711
[481]	validation_0-logloss:0.10706
[482]	validation_0-logloss:0.10705
[483]	validation_0-logloss:0.10708
[484]	validation_0-logloss:0.10712
[485]	validation_0-logloss:0.10715
[486]	validation_0-logloss:0.10717
[487]	validation_0-logloss:0.10721
[488]	validation_0-logloss:0.10724
[489]	validation_0-logloss:0.10727
[490]	validation_0-logloss:0.10730
[491]	validation_0-logloss:0.10730
[492]	validation_0-logloss:0.10731
[493]	validation_0-logloss:0.10733
[494]	validation_0-logloss:0.10734
[495]	validation_0-logloss:0.10736
[496]	validation_0-logloss:0.10739
[497]	validation_0-logloss:0.10741
[498]	validation_0-logloss:0.10740
[499]	validation_0-logloss:0.10740
[500]	validation_0-logloss:0.10741
[501]	validation_0-logloss:0.10745
[502]	validation_0-logloss:0.10744
[503]	validation_0-logloss:0.10742
[504]	validation_0-l